In [28]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import roc_curve
import pandas as pd

In [7]:
# Define transforms for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Set the paths to your training and validation directories
train_dir = '/kaggle/input/morph-splitted/train'
val_dir = '/kaggle/input/morph-splitted/val'
test_dir = '/kaggle/input/morph-splitted/test'

In [8]:
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val']),
    'test': datasets.ImageFolder(test_dir, data_transforms['test'])
}


In [17]:
# Create data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=512, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=512, shuffle=False, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=512, shuffle=False, num_workers=4)
}


In [11]:
# Define the CNN model for binary classification
class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 1)  # Output layer with 1 neuron for binary classification
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
# Initialize the model, loss function, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BinaryCNN().to(device)
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Training and evaluation loop
num_epochs = 10
best_model_wts = model.state_dict()
best_acc = 0.0


In [14]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [15]:
# Number of epochs
num_epochs = 10
best_acc = 0.0
best_model_wts = model.state_dict()

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()  # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data with progress bar
        with tqdm(total=len(dataloaders[phase]), desc=f'{phase} Phase', unit='batch') as pbar:
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.float().view(-1, 1).to(device)  # Reshape labels for binary classification

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.sigmoid(outputs) > 0.5
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Update progress bar
                pbar.update(1)
                pbar.set_postfix(loss=running_loss / (pbar.n * inputs.size(0)),
                                 accuracy=running_corrects.double() / (pbar.n * inputs.size(0)))

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()

    print()

# Load best model weights
model.load_state_dict(best_model_wts)
print('Best val Acc: {:4f}'.format(best_acc))


Epoch 1/10
----------


train Phase: 100%|██████████| 47/47 [01:00<00:00,  1.30s/batch, accuracy=tensor(1.0356, device='cuda:0', dtype=torch.float64), loss=0.267]


train Loss: 0.2338 Acc: 0.9086


val Phase: 100%|██████████| 16/16 [00:26<00:00,  1.68s/batch, accuracy=tensor(1.5295, device='cuda:0', dtype=torch.float64), loss=0.103] 


val Loss: 0.0660 Acc: 0.9789

Epoch 2/10
----------


...

val Loss: 0.0413 Acc: 0.9859

Epoch 10/10
----------


train Phase: 100%|██████████| 47/47 [01:02<00:00,  1.33s/batch, accuracy=tensor(1.1116, device='cuda:0', dtype=torch.float64), loss=0.0786]


train Loss: 0.0690 Acc: 0.9752


val Phase: 100%|██████████| 16/16 [00:15<00:00,  1.03batch/s, accuracy=tensor(1.5371, device='cuda:0', dtype=torch.float64), loss=0.074] 

val Loss: 0.0474 Acc: 0.9838

Best val Acc: 0.986000


In [18]:
# Test the model
model.eval()
running_loss = 0.0
running_corrects = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.float().view(-1, 1).to(device)

        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

test_loss = running_loss / len(image_datasets['test'])
test_acc = running_corrects.double() / len(image_datasets['test'])

print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}')


Test Loss: 0.0474 Acc: 0.9823


In [19]:
import torch
from torch.utils.data import DataLoader

# Function to create data loaders
def create_data_loader(data_dir, transform, batch_size):
    dataset = datasets.ImageFolder(data_dir, transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return data_loader

# Define the transforms for the datasets
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 512

# Create data loaders for each dataset
fm_loader = create_data_loader('/kaggle/input/mad-benchmark/FaceMorpher', transform, batch_size)
mg1_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_I', transform, batch_size)
mg2_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_II', transform, batch_size)
oc_loader = create_data_loader('/kaggle/input/mad-benchmark/OpenCV', transform, batch_size)
wm_loader = create_data_loader('/kaggle/input/mad-benchmark/Webmorph', transform, batch_size)


In [20]:
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
results = []
losses = []

# Evaluate the model on each dataset
criterion = nn.BCEWithLogitsLoss()
model.eval()

BinaryCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=50176, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [21]:
for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.float().view(-1, 1).to(device)

        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    test_loss = running_loss / total_samples
    test_accuracy = running_corrects.double() / total_samples

    results.append(test_accuracy.item())
    losses.append(test_loss)

    print(f"{dataset_name} - Loss: {test_loss:.4f} Acc: {test_accuracy:.4f}")

# Print the final results
print("\nFinal Results:")
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
for name, accuracy in zip(names, results):
    print(f"{name}: {accuracy:.4f}")


Evaluating dataset: FaceMorpher


FaceMorpher - Loss: 0.8286 Acc: 0.7342
Evaluating dataset: MIPGAN_I


MIPGAN_I - Loss: 0.9904 Acc: 0.6811
Evaluating dataset: MIPGAN_II


MIPGAN_II - Loss: 0.7850 Acc: 0.7273
Evaluating dataset: OpenCV


OpenCV - Loss: 6.8563 Acc: 0.1759
Evaluating dataset: Webmorph


Webmorph - Loss: 7.1050 Acc: 0.2898

Final Results:
FaceMorpher: 0.7342
MIPGAN_I: 0.6811
MIPGAN_II: 0.7273
OpenCV: 0.1759
Webmorph: 0.2898


### **Evaluation Metrics**

In [30]:
import torch
import numpy as np
from sklearn.metrics import roc_curve

def calculate_metrics(true_labels, predictions, fixed_bpcer=None, fixed_apcer=None):
    """Calculate APCER, BPCER, EER, and their corresponding thresholds."""
    # Convert true_labels and predictions to PyTorch tensors
    true_labels = torch.tensor(true_labels)
    predictions = torch.tensor(predictions)
    
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Calculate APCER and its threshold at fixed BPCER
    if fixed_bpcer is not None:
        fpr_target = fixed_bpcer
        closest_fpr_index = torch.argmin(torch.abs(torch.tensor(fpr) - fpr_target))
        apcer = 1 - tpr[closest_fpr_index]
        apcer_threshold = thresholds[closest_fpr_index]
    else:
        apcer = apcer_threshold = None
    
    # Calculate BPCER and its threshold at fixed APCER
    if fixed_apcer is not None:
        tpr_target = 1 - fixed_apcer
        closest_tpr_index = torch.argmin(torch.abs(torch.tensor(tpr) - tpr_target))
        bpcer = fpr[closest_tpr_index]
        bpcer_threshold = thresholds[closest_tpr_index]
    else:
        bpcer = bpcer_threshold = None
    
    # Calculate EER and its threshold
    frr = 1 - tpr
    eer_index = torch.argmin(torch.abs(torch.tensor(fpr) - torch.tensor(frr)))
    eer = fpr[eer_index]
    eer_threshold = thresholds[eer_index]
    
    return {
        "APCER": apcer,
#         "APCER Threshold": apcer_threshold,
        "BPCER": bpcer,
#         "BPCER Threshold": bpcer_threshold,
        "EER": eer,
#         "EER Threshold": eer_threshold
    }

# Define datasets and model predictions
datasets = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_bpcer_values = [0.01, 0.1, 0.2]
fixed_apcer_values = [0.01, 0.1, 0.2]
all_results = []

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    print(f"Evaluating model on dataset: {name}")
    
    # Predictions and true labels
    all_predictions = []
    all_true_labels = []
    for inputs, labels in dataset:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU
        predictions = model(inputs)
        all_predictions.append(predictions.detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Calculate metrics for each fixed BPCER
    for fixed_bpcer in fixed_bpcer_values:
        print(f"Calculating metrics for fixed BPCER: {fixed_bpcer}")
        metrics = calculate_metrics(true_labels, predictions, fixed_bpcer=fixed_bpcer)
        result = {
            "Dataset": name,
            "Fixed BPCER": f"{fixed_bpcer * 100:.1f}%",
            **metrics
        }
        all_results.append(result)
    
    # Calculate metrics for each fixed APCER
    for fixed_apcer in fixed_apcer_values:
        print(f"Calculating metrics for fixed APCER: {fixed_apcer}")
        metrics = calculate_metrics(true_labels, predictions, fixed_apcer=fixed_apcer)
        result = {
            "Dataset": name,
            "Fixed APCER": f"{fixed_apcer * 100:.1f}%",
            **metrics
        }
        all_results.append(result)

# Convert the results to a Pandas DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


Evaluating model on dataset: FaceMorpher
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_I
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_II
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: OpenCV
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics

In [ ]:
# Save the DataFrame to a CSV file
df_results.to_csv('metrics_results.csv', index=False)


In [31]:
import torch
import numpy as np
from sklearn.metrics import roc_curve

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    """Calculate APCER at a fixed BPCER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    """Calculate BPCER at a fixed APCER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    """Calculate EER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

# Define datasets and model predictions
datasets = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_bpcer_values = [0.01, 0.1, 0.2]
fixed_apcer_values = [0.01, 0.1, 0.2]
all_results = []

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    print(f"Evaluating model on dataset: {name}")
    
    # Predictions and true labels
    all_predictions = []
    all_true_labels = []
    for inputs, labels in dataset:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU
        predictions = model(inputs)
        all_predictions.append(predictions.detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Calculate metrics for each fixed BPCER
    for fixed_bpcer in fixed_bpcer_values:
        print(f"Calculating metrics for fixed BPCER: {fixed_bpcer}")
        apcer = calculate_apcer(true_labels, predictions, fixed_bpcer)
        result = {
            "Dataset": name,
            "Fixed BPCER": f"{fixed_bpcer * 100:.1f}%",
            "APCER": apcer
        }
        all_results.append(result)
    
    # Calculate metrics for each fixed APCER
    for fixed_apcer in fixed_apcer_values:
        print(f"Calculating metrics for fixed APCER: {fixed_apcer}")
        bpcer = calculate_bpcer(true_labels, predictions, fixed_apcer)
        result = {
            "Dataset": name,
            "Fixed APCER": f"{fixed_apcer * 100:.1f}%",
            "BPCER": bpcer
        }
        all_results.append(result)

    # Calculate EER
    eer = calculate_eer(true_labels, predictions)
    result = {
        "Dataset": name,
        "EER": eer
    }
    all_results.append(result)

# Convert the results to a Pandas DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


Evaluating model on dataset: FaceMorpher
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_I
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_II
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: OpenCV
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics